## Import datasetu

In [16]:
import deepchem as dc
import rdkit
import polars as pl
import numpy as np


In [2]:
dc.__version__

'2.6.0.dev'

## Przegląd danych

In [8]:
tasks, datasets, transformers = dc.molnet.load_chembl(
    featurizer=dc.feat.MACCSKeysFingerprint(),
    set='sparse',
    reload=False
)

Łączenie danych w jeden dataset

In [12]:
train_dataset, valid_dataset, test_dataset = datasets

full_dataset = dc.data.DiskDataset.merge(datasets)

In [14]:
# Pobranie nazw zadań (tasks)
tasks = full_dataset.get_task_names()

# Ekstrakcja danych NumPy z obiektu DeepChem Dataset
X_data = full_dataset.X
Y_data = full_dataset.y
W_data = full_dataset.w
mol_ids = full_dataset.ids

In [17]:
# Użycie maski wag (W) do stworzenia maski logicznej
# Tam, gdzie W=0.0, wartość Y jest nieznana.
# Użyjemy tej maski, aby jawnie ustawić NaN.
unknown_mask = (W_data == 0.0)

# Kopia danych Y, gdzie nieznane wartości są ustawione na NaN
Y_data_masked = Y_data.copy()
Y_data_masked[unknown_mask] = np.nan

# 1. Tworzenie Polars DataFrame z Y_data_masked
# Nazwy kolumn bierzemy z listy tasks
df_chembl_y = pl.DataFrame(
    {task: Y_data_masked[:, i] for i, task in enumerate(tasks)}
)

# 2. Dodanie identyfikatorów
df_chembl_y = df_chembl_y.with_columns(
    pl.Series("ID", mol_ids)
)

print("\n--- Fragment Data Frame Polars (Y) ---\n")
print(df_chembl_y.head())


--- Fragment Data Frame Polars (Y) ---

shape: (5, 692)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ CHEMBL107 ┆ CHEMBL107 ┆ CHEMBL107 ┆ CHEMBL107 ┆ … ┆ CHEMBL616 ┆ CHEMBL616 ┆ CHEMBL618 ┆ ID       │
│ 5051      ┆ 5104      ┆ 5145      ┆ 5189      ┆   ┆ 4         ┆ 6         ┆ 4         ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ str      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ NaN       ┆ NaN       ┆ NaN       ┆ NaN       ┆ … ┆ NaN       ┆ NaN       ┆ NaN       ┆ CC(C)C[C │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ @H](NC(= │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ O)[C@H]( │
│           ┆           ┆         

In [18]:
# Utworzenie Polars DataFrame z cechami (X)
# Nadanie kolumnom generycznych nazw (MACCS_0, MACCS_1, ...)
feature_names = [f"MACCS_{i}" for i in range(X_data.shape[1])]
df_chembl_x = pl.DataFrame(
    X_data,
    schema=feature_names
)
df_chembl_x = df_chembl_x.with_columns(
    pl.Series("ID", mol_ids)
)

print("\n--- Fragment Data Frame Polars (X) - Cechy MACCS ---\n")
print(df_chembl_x.head())


--- Fragment Data Frame Polars (X) - Cechy MACCS ---

shape: (5, 168)
┌─────────┬─────────┬─────────┬─────────┬───┬───────────┬───────────┬───────────┬──────────────────┐
│ MACCS_0 ┆ MACCS_1 ┆ MACCS_2 ┆ MACCS_3 ┆ … ┆ MACCS_164 ┆ MACCS_165 ┆ MACCS_166 ┆ ID               │
│ ---     ┆ ---     ┆ ---     ┆ ---     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---              │
│ i64     ┆ i64     ┆ i64     ┆ i64     ┆   ┆ i64       ┆ i64       ┆ i64       ┆ str              │
╞═════════╪═════════╪═════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪══════════════════╡
│ 0       ┆ 0       ┆ 0       ┆ 0       ┆ … ┆ 1         ┆ 1         ┆ 0         ┆ CC(C)C[C@H](NC(= │
│         ┆         ┆         ┆         ┆   ┆           ┆           ┆           ┆ O)[C@H](CS(=O)…  │
│ 0       ┆ 0       ┆ 0       ┆ 0       ┆ … ┆ 1         ┆ 1         ┆ 0         ┆ C[C@H](\C=C(/C)\ │
│         ┆         ┆         ┆         ┆   ┆           ┆           ┆           ┆ C=C\C(=O)NO)C(…  │
│ 0       ┆ 0       